In [2]:
import json
import os.path
import pathlib
# import bdgd_tools
import time
# from bdgd_tools import Sample, Case, Circuit, LineCode
import geopandas as gpd
import re

class JsonData:
    def __init__(self, file_name):
        """
        Inicializa a classe JsonData com o nome do arquivo de entrada.

        :param file_name: Nome do arquivo JSON de entrada.
        """
        self.data = self._read_json_file(file_name)
        self.tables = self._create_tables()

    @staticmethod
    def _read_json_file(file_name):
        """
        Lê o arquivo JSON fornecido e retorna o conteúdo como um objeto Python.

        :param file_name: Nome do arquivo JSON de entrada.
        :return: Objeto Python contendo o conteúdo do arquivo JSON.
        """
        with open(file_name, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data

    def _create_tables(self):
        """
        Cria um dicionário de tabelas a partir dos dados carregados do arquivo JSON.

        :return: Dicionário contendo informações das tabelas a serem processadas.
        """
        return {
            table_name: Table(
                table_name,
                table_data["columns"],
                table_data["type"],
                table_data["ignore_geometry"],
            )
            for table_name, table_data in self.data["configuration"][
                "tables"
            ].items()
        }

    def get_tables(self):
        """
        Retorna o dicionário de tabelas.

        :return: Dicionário contendo informações das tabelas a serem processadas.
        """
        return self.tables

    @staticmethod
    def convert_data_types(df, column_types):
        """
        Converte os tipos de dados das colunas do DataFrame fornecido.

        :param df: DataFrame a ser processado.
        :param column_types: Dicionário contendo mapeamento de colunas para tipos de dados.
        :return: DataFrame com tipos de dados convertidos.
        """
        return df.astype(column_types)

    def create_geodataframes(self, file_name, runs=1):
        """
        Cria GeoDataFrames a partir de um arquivo de entrada e coleta estatísticas.

        :param file_name: Nome do arquivo de entrada.
        :param runs: Número de vezes que cada tabela será carregada e convertida (padrão: 1).
        :return: Dicionário contendo GeoDataFrames e estatísticas.
        """
        geodataframes = {}

        for table_name, table in self.tables.items():
            load_times = []
            conversion_times = []
            gdf_converted = None

            for _ in range(runs):
                start_time = time.time()
                gdf_ = gpd.read_file(file_name, layer=table.name, include_fields=table.columns,
                                     ignore_geometry=table.ignore_geometry)
                start_conversion_time = time.time()
                gdf_converted = self.convert_data_types(gdf_, table.data_types)                    
                end_time = time.time()
 
                load_times.append(start_conversion_time - start_time)
                conversion_times.append(end_time - start_conversion_time)

            load_time_avg = sum(load_times) / len(load_times)
            conversion_time_avg = sum(conversion_times) / len(conversion_times)
            mem_usage = gdf_converted.memory_usage(index=True, deep=True).sum() / 1024 ** 2

            geodataframes[table_name] = {
                'gdf': gdf_converted,
                'memory_usage': mem_usage,
                'load_time_avg': load_time_avg,
                'conversion_time_avg': conversion_time_avg,
                'ignore_geometry': table.ignore_geometry
            }
        return geodataframes




In [3]:
import pandas as pd

class Table:
    def __init__(self, name, columns, data_types, ignore_geometry_):
        self.name = name
        self.columns = columns
        self.data_types = data_types
        self.ignore_geometry = ignore_geometry_

    def __str__(self):
        return f"Table(name={self.name}, columns={self.columns}, data_types={self.data_types}, " \
               f"ignore_geometry={self.ignore_geometry})"


In [4]:
def merge_entities_tables(dataframe1: gpd.geodataframe.GeoDataFrame,dataframe2: gpd.geodataframe.GeoDataFrame):
    """
    Merge two GeoDataFrames of entities based on their indices and handle duplicated columns. 
    
    It's necessary when the element needs more of one table of the BDGD. 

    Parameters:
    dataframe1 (gpd.geodataframe.GeoDataFrame): The first GeoDataFrame (entity table) to be merged.
    dataframe2 (gpd.geodataframe.GeoDataFrame): The second GeoDataFrame (entity table) to be merged.

    Returns:
    gpd.geodataframe.GeoDataFrame: The merged GeoDataFrame with duplicated columns removed.

    """

    merged_dfs= dataframe2.join(dataframe1, lsuffix='_left')
    duplicated_columns = [col for col in merged_dfs.columns if '_left' in col]
    merged_dfs.drop(columns=duplicated_columns, inplace=True)

    return  merged_dfs

In [5]:
def inner_entities_tables(entity1_df, enetity2_df, left_column: str = "", right_column: str = ""):
    
    """
    Merge two entities's DataFrames using an inner join and process the resulting DataFrame.

    This function takes two DataFrames, 'entity1_df' and 'entity2_df', and merges them
    using an inner join on the 'UN_RE' column of 'entity1_df' and the 'COD_ID' column
    of 'entity2_df'. The resulting merged DataFrame is then processed to remove
    redundant columns and rename columns as necessary.

    Parameters:
    - entity1_df (pandas.DataFrame): The first DataFrame to be merged. It belongs to an entity A. 
    - entity2_df (pandas.DataFrame): The second DataFrame to be merged. It belongs to an entity B.

    Returns:
    - pandas.DataFrame: A new DataFrame 

    Example:
    entity1_df:
        UN_RE  POT_NOM_x  PAC1_x
        A     Value1     Value2
        B     Value3     Value4

    entity2_df:
        COD_ID  POT_NOM_y  PAC3_y
        A      Value5     Value6
        C      Value7     Value8

    Resulting DataFrame after calling inner_entities_tables(entity1_df, entity2_df):
        UN_RE  POT_NOM  CPOT_NOM  COD_ID  PAC3
        A      Value1    Value2       A     Value6

    Note:
    - Columns with '_x' suffix are dropped.
    - Columns with '_y' suffix have the suffix removed.

    """

    merged_dfs = pd.merge( entity1_df, enetity2_df,left_on='UN_RE', right_on='COD_ID', how='inner')

    for column in merged_dfs.columns:
        if column.endswith('_x'):
            merged_dfs.drop(columns=column, inplace=True)
        elif column.endswith('_y'):
            new_column_name = column[:-2]  # Remove the '_df2' suffix
            merged_dfs.rename(columns={column: new_column_name}, inplace=True)

    return merged_dfs

In [6]:
json_file_name = os.path.join(os.getcwd(), "bdgd2dss.json")

json_data = JsonData(json_file_name)

In [7]:
# geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\sample\raw\aneel\muxenergia.gdb")
# geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\sample\raw\aneel\muxenergia.gdb")
# geodataframes2 = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\ENEL_CE_39_2021-12-31_V10_20230124-0859.gdb")

geodataframes = json_data.create_geodataframes(r"C:\Users\ADERALDO\Desktop\TCC BDGB\CRELUZ-D_598_2022-12-31_V11_20230831-0921.gdb")

In [8]:
alimentador = '1_3PAS_1'
df = geodataframes["UCBT_tab"]['gdf'].query("CTMT == @alimentador")
# df_eqtrmt = geodataframes['EQTRMT']['gdf']
# df = df[df["COD_ID"] == "7107"]
# if 'IP' in df["COD_ID"].unique():
#     print('cu')
df = df["PAC"]
df


75       UC2832903
111      UC2832902
517      UC2832901
519      UC2832709
561      UC2832708
           ...    
25113    UC2832910
25126    UC2832909
25142    UC2832907
25152    UC2832906
25338    UC2832904
Name: PAC, Length: 432, dtype: object

In [9]:
def verificar_string_em_arquivo(nome_arquivo, texto):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            conteudo = arquivo.read()
            if texto in conteudo:
                return f''
            else:
                return f'A string "{texto}" não está presente no arquivo.'
    except FileNotFoundError:
        return f'O arquivo "{nome_arquivo}" não foi encontrado.'
    except Exception as e:
        return f'Ocorreu um erro: {e}'

# Exemplo de uso da função:
nome_do_arquivo = 'exemplo.txt'
texto_procurado = 'mae'

resultado = verificar_string_em_arquivo(nome_do_arquivo, texto_procurado)
print(resultado)

A string "mae" não está presente no arquivo.


In [13]:
for i in df:
    resultado = verificar_string_em_arquivo(nome_do_arquivo, i)
    print(resultado)




































A string "UC2830104" não está presente no arquivo.















































A string "UC2819369" não está presente no arquivo.









A string "UC2806086" não está presente no arquivo.











A string "UC2818346" não está presente no arquivo.










































A string "UC2805385" não está presente no arquivo.














A string "UC2820646" não está presente no arquivo.


A string "UC2805363" não está presente no arquivo.
















































































A string "UC2818313" não está presente no arquivo.












































A string "UC2824956" não está presente no arquivo.





























A string "UC2805317" não está presente no arquivo.







































































A string "UC2805361" não está presente no arquivo.







































In [33]:
alimentador = '1_3PAS_1'
df_untrmt = df.query("CTMT==@alimentador")
df_untrmt

,COD_ID,PAC_1,PAC_2,PAC_3,FAS_CON_P,FAS_CON_S,FAS_CON_T,TAP,POT_NOM,PER_FER,PER_TOT,CTMT,TIP_TRAFO,MRT
38,28_6216,47101,ET28_6216,0,ABC,ABCN,0,1.0,30.0,160.0,790.0,1_3PAS_1,T,0
127,28_630,2931,ET28_630,ET28_630,B,ABN,0,1.0,25.0,90.0,480.0,1_3PAS_1,MT,1
172,28_6193,799581,ET28_6193,ET28_6193,A,ABN,0,1.0,25.0,90.0,480.0,1_3PAS_1,MT,1
177,28_6194,797861,ET28_6194,ET28_6194,A,ABN,0,1.0,25.0,90.0,480.0,1_3PAS_1,MT,1
191,28_5321,741991,ET28_5321,ET28_5321,A,ABN,0,1.0,25.0,90.0,480.0,1_3PAS_1,MT,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5821,28_4047,12151,ET28_4047,ET28_4047,A,ABN,0,1.0,15.0,65.0,330.0,1_3PAS_1,MT,1
5823,28_6254,805561,ET28_6254,0,ABC,ABCN,0,1.0,30.0,160.0,790.0,1_3PAS_1,T,0
5824,28_6255,2991,ET28_6255,ET28_6255,A,ABN,0,1.0,10.0,50.0,245.0,1_3PAS_1,MT,1
5825,28_6256,805473,ET28_6256,ET28_6256,A,ABN,0,1.0,10.0,50.0,245.0,1_3PAS_1,MT,1


In [36]:
# df_result = merge_entities_tables(df_untrmt, df_eqtrmt)
df_result = inner_entities_tables(df_eqtrmt, df_untrmt, left_column='UNI_TR_MT', right_column='COD_ID')
df_result

,FAS_CON,TEN_PRI,TEN_SEC,TEN_TER,LIG_FAS_P,LIG_FAS_S,LIG_FAS_T,UNI_TR_MT,COD_ID,PAC_1,...,FAS_CON_P,FAS_CON_S,FAS_CON_T,TAP,POT_NOM,PER_FER,PER_TOT,CTMT,TIP_TRAFO,MRT
0,ABC,49,10,0,B,AN,0,28_3654,28_3654,11151,...,B,AN,0,1.0,5.0,35.0,140.0,1_3PAS_1,M,1
1,ABC,62,17,0,CA,AN,BN,28_633,28_633,74631,...,CA,ABN,0,1.0,25.0,100.0,520.0,1_3PAS_1,MT,0
2,C,49,17,0,C,AN,BN,28_3311,28_3311,42861,...,C,ABN,0,1.0,10.0,50.0,245.0,1_3PAS_1,MT,1
3,A,49,17,0,A,AN,BN,28_4958,28_4958,47891,...,A,ABN,0,1.0,15.0,65.0,330.0,1_3PAS_1,MT,1
4,ABC,62,15,0,ABC,ABCN,0,28_1990,28_1990,325621,...,ABC,ABCN,0,1.0,75.0,725.0,3605.0,1_3PAS_1,T,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,B,49,17,0,B,AN,BN,28_2247,28_2247,134801,...,B,ABN,0,1.0,10.0,50.0,245.0,1_3PAS_1,MT,1
131,A,49,17,0,A,AN,BN,28_2288,28_2288,357411,...,A,ABN,0,1.0,5.0,35.0,140.0,1_3PAS_1,MT,1
132,ABC,62,17,0,CA,AN,BN,28_632,28_632,72501,...,CA,ABN,0,1.0,15.0,75.0,365.0,1_3PAS_1,MT,0
133,ABC,62,15,0,ABC,ABCN,0,28_4146,28_4146,645251,...,ABC,ABCN,0,1.0,75.0,725.0,3605.0,1_3PAS_1,T,0


In [7]:
def convert_tfascon_bus_prim(case):
    switch_dict = {
        'A': '1',
        'B': '2',
        'C': '3',
        'AN': '1.0',
        'BN': '2.0',
        'CN': '3.0',
        'AB': '1.2',
        'BC': '2.3',
        'CA': '3.1',
        'ABN': '1.2.0',
        'BCN': '2.3.0',
        'CAN': '3.1.0',
        'ABC': '1.2.3',
        'ABCN': '1.2.3.0',
    }
    return switch_dict.get(case, 'Invalid case')


In [8]:
convert_tfascon_bus_prim('B')

'2'

In [5]:
nom_alimentador = '2_TAP2_1'
# alimentador = "CTMT=={nom_alimentador}"
df = geodataframes["UCBT_tab"]['gdf']
var = df.query("CTMT==@nom_alimentador")
# df
var

,PAC,PN_CON,CTMT,TIP_CC,FAS_CON,TEN_FORN
2,UC14865,1284,2_TAP2_1,RES_2,BCN,15
3,UC15172,7585,2_TAP2_1,RES_2,AN,10
7,UC15176,1321,2_TAP2_1,COM_13,ABCN,15
10,UC14865,1284,2_TAP2_1,RES_2,CAN,15
12,UC20061,7553,2_TAP2_1,RES_7,ABCN,15
...,...,...,...,...,...,...
12336,UC321,1341,2_TAP2_1,RES_2,CN,10
12339,UC328,164,2_TAP2_1,RES_11,AN,10
12347,UC738,1975,2_TAP2_1,RES_2,CN,10
12350,UC8812,71,2_TAP2_1,RES_2,CN,10


In [23]:
var = "cu"
if not os.path.exists(f"sample/{var}"):
    os.mkdir(f'sample/{var}')

output_directory= os.path.join(os.getcwd(), f'output/{feeder}')


In [25]:
output_directory= os.path.join(os.getcwd(), f'output\{var}')
print(output_directory)


c:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\bdgd_tools\output\cu


In [32]:
def rename_linecode_string(i, input_str: str) -> str:
    """
    This function re-writes the string identfying key places by specified parameters and insering caracteres.  

    Args: 

    Returns: 

    In this case, it should modify the names of line, bus1, bus2 and linecode. 

    """
    pattern = r'Cargas_BT_(\d+)_1_TAP2_1.dss'  

    def repl(match):
        linecode_num = match.group(1)
        print(f'New "Linecode.{linecode_num}') 

    
    re.sub(pattern, repl, input_str)

In [33]:
rename_linecode_string(i= '3', input_str='Cargas_BT_DU02_1_TAP2_1.dss')

In [56]:
file_names = ['CircuitoMT_1_TAP2_1.dss', 'Arranjos_1_TAP2_1.dss', 'Trecho_MT_1_TAP2_1.dss', 'Trecho_BT_1_TAP2_1.dss', 'Ramais_BT_1_TAP2_1.dss', 'Regulador_Tensao_1_TAP2_1.dss', 'Transformador_MTBT_1_TAP2_1.dss', 'CurvaCarga_1_TAP2_1.dss', 'Cargas_BT_DU02_1_TAP2_1.dss', 'Cargas_MT_DU02_1_TAP2_1.dss'] 
for elemento in file_names:
    if "Cargas" in elemento:
        indice = file_names.index(elemento)

indice

9

In [67]:
def create_outputs_masters(file_names):
    
    
    for elemento in file_names:
        if "Cargas" in elemento:
            indice = file_names.index(elemento)

    base_string_BT = file_names[indice - 1]
    base_string_MT = file_names[indice]

    for i in ['DU', 'SA', 'DO']:
        file_names[indice-1] = base_string_BT.replace('_DU', f'_{i}')
        file_names[indice] = base_string_MT.replace('_DU', f'_{i}')
        


In [69]:
create_outputs_masters(file_names=file_names)

['CircuitoMT_1_TAP2_1.dss', 'Arranjos_1_TAP2_1.dss', 'Trecho_MT_1_TAP2_1.dss', 'Trecho_BT_1_TAP2_1.dss', 'Ramais_BT_1_TAP2_1.dss', 'Regulador_Tensao_1_TAP2_1.dss', 'Transformador_MTBT_1_TAP2_1.dss', 'CurvaCarga_1_TAP2_1.dss', 'Cargas_BT_DO02_1_TAP2_1.dss', 'Cargas_MT_DO02_1_TAP2_1.dss']


In [73]:
a = "ola"
b = "hello"

a, b = b, a
a

'hello'

In [78]:
def stringa():
    return f'ola'

print(f'{stringa()}')

ola


In [ ]:
json_file_name = os.path.join(os.getcwd(), "bdgd2dss.json")

json_data = JsonData(json_file_name)

In [3]:
import os

pasta = r'C:\Users\ADERALDO\Desktop\TCC BDGB\BDGD_Dev\bdgd-tools\output\1_3PAS_1'

with open('lista_arquivos.txt', 'w') as f:
    lista_arquivos = os.listdir(pasta)
    for arquivo in lista_arquivos:
        f.write(f"{arquivo}\n")